<a href="https://colab.research.google.com/github/Vinayak-Sharma12/Assignment/blob/main/ThaparGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install python-docx
!pip install python-pptx
!pip install PyPDF2
!pip install langchain
!pip install langchain_community
!pip install langchain_google_genai
!pip install langchain_text_splitters
!pip install sentence-transformers
!pip install faiss-cpu
!pip install cohere

In [51]:
# necessary Imports
from docx import Document
from PyPDF2 import PdfReader
from pptx import Presentation
from langchain_community.llms import Cohere
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts  import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

In [55]:
#rb means open the file in binary mode which will not give error in text extract
pdf_file = open("/content/Thapar_Institute_of_Engineering_and_Technology.pdf",'rb')
# when you opem a text file 'r' format interprets these bytes according to the system's character encoding (like UTF-8 or ASCII).
#when you open it in rb format it stores data in raw byte format




In [56]:
# extracting pdf data
pdf_text = ""
pdf_reader = PdfReader(pdf_file)
for page in pdf_reader.pages:
    pdf_text += page.extract_text()

# CHUNKING

In [57]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 200, # This is helpul to handle the data loss while chunking.
        length_function = len,
        separators=['\n', '\n\n', ' ', '']
    )

In [58]:
chunks = text_splitter.split_text(text = pdf_text)

In [59]:
len(chunks)   #Total number of chunks created after text split

7

# Embedding and Storing it aka INDEXING

In [60]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [61]:
vectorstore = FAISS.from_texts(chunks, embedding = embeddings)

# Retriever

In [62]:
# creating retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})


In [63]:
retrieved_docs = retriever.invoke("Companies that come in Thapar?")

In [64]:
len(retrieved_docs)

5

# PROMPTING

In [66]:
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                not contained in the context, say "answer not available in context" \n\n
                Context: \n {context}?\n
                Question: \n {question} \n
                Answer:"""

prompt = PromptTemplate.from_template(template=prompt_template)

In [67]:
# function to create a single string of relevant documents given by Faiss.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [77]:
import os

In [69]:
# RAG Chain

def generate_answer(question):
    cohere_llm = Cohere(model="command", temperature=0.1, cohere_api_key='sRmFY97EVTJa7VaaaQha5oH7lScl1rxTZv8x6KrV')

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | cohere_llm
        | StrOutputParser()
    )

    return rag_chain.invoke(question)

In [75]:
ans = generate_answer("When was Thapar Established")

In [71]:
ans = generate_answer("What is the percentage of placement of each branch in Thapar")

In [76]:
print(ans)

 Thapar Institute of Engineering and Technology was established in 1956. 
